In [1]:
# Преземи верзија од GitLab на македонскиот текст-во-говор (ТГ) модел Македонка
!git clone https://github.com/f-data/TTS-Makedonka.git

Cloning into 'TTS-Makedonka'...
remote: Enumerating objects: 524, done.
remote: Counting objects: 100% (524/524), done.
remote: Compressing objects: 100% (505/505), done.
remote: Total 524 (delta 14), reused 519 (delta 12), pack-reused 0
Receiving objects: 100% (524/524), 49.27 MiB | 29.94 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [ ]:
# Преземи го ТГ моделот
!gdown https://drive.google.com/uc?id=1XjxpHhDOG0B1LHO2pVKI_51sSEd_eMvv

In [48]:
# инсталација на потребните зависности за работа на DeepVoice 3 моделот

!pip install tensorflow==1.13.0rc1
!pip install unidecode
!pip install docopt
!pip install tqdm
!pip install nnmnkwii
!pip install requests
!pip install lws

remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 1), reused 3 (delta 1), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/f-data/TTS-Makedonka
   2ead0cb..36751ec  master     -> origin/master
Updating 2ead0cb..36751ec
Fast-forward
 synthesis.py | 168 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 168 insertions(+)
 create mode 100644 synthesis.py


In [3]:
cd TTS-Makedonka/

/content/TTS-Makedonka


In [4]:
import hashlib
import re
import sys
import os
from os.path import dirname, join, basename, splitext
import torch
import numpy as np
import nltk

# The deepvoice3 model
from deepvoice3_pytorch import frontend
from hparams import hparams, hparams_debug_string

from tqdm import tqdm
import nltk
nltk.download('cmudict')
nltk.download('punkt')
import audio
import IPython
from IPython.display import Audio, display

In [35]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
_frontend = None  # to be set later

In [18]:
# ф-ја за вчитување на претренираната инстанца од моделот
def load_checkpoint(checkpoint_path):
    if use_cuda:
        checkpoint = torch.load(checkpoint_path)
    else:
        checkpoint = torch.load(checkpoint_path,
                                map_location=lambda storage, loc: storage)
    return checkpoint

In [57]:
#ф-ја за конверзија на текст-во-говор со користење на моделот
def tts(model, text, p=0, speaker_id=None, fast=False):
    """Convert text to speech waveform given a deepvoice3 model.

    Args:
        text (str) : Input text to be synthesized
        p (float) : Replace word to pronounciation if p > 0. Default is 0.
    """
    sequence = np.array(_frontend.text_to_sequence(text, p=0))
    sequence = torch.from_numpy(sequence).unsqueeze(0).long().to(device)
    text_positions = torch.arange(1, sequence.size(-1) + 1).unsqueeze(0).long().to(device)
    speaker_ids = None if speaker_id is None else torch.LongTensor([speaker_id]).to(device)

    # Greedy decoding
    with torch.no_grad():
        mel_outputs, linear_outputs, alignments, done = model(
            sequence, text_positions=text_positions, speaker_ids=speaker_ids)

    linear_output = linear_outputs[0].cpu().data.numpy()
    spectrogram = audio._denormalize(linear_output)
    alignment = alignments[0].cpu().data.numpy()
    mel = mel_outputs[0].cpu().data.numpy()
    mel = audio._denormalize(mel)

    # Predicted audio signal
    waveform = audio.inv_spectrogram(linear_output.T)

    return waveform, alignment, spectrogram, mel
  


In [ ]:
#визуелизација на порамнувањето и спектограмот
def visualize(alignment, spectrogram):
  label_fontsize = 16
  figure(figsize=(16,16))

  subplot(2,1,1)
  imshow(alignment.T, aspect="auto", origin="lower", interpolation=None)
  xlabel("Decoder timestamp", fontsize=label_fontsize)
  ylabel("Encoder timestamp", fontsize=label_fontsize)
  colorbar()

  subplot(2,1,2)
  librosa.display.specshow(spectrogram.T, sr=fs, 
                           hop_length=hop_length, x_axis="time", y_axis="linear")
  xlabel("Time", fontsize=label_fontsize)
  ylabel("Hz", fontsize=label_fontsize)
  tight_layout()
  colorbar()

In [42]:
# вчитување на чекпоинтот од Македонка моделот

checkpoint_path = '/content/TTS-Makedonka/checkpoint_step000620000.pth' 
from train import plot_alignment, build_model
_frontend = getattr(frontend, hparams.frontend)

In [45]:
checkpoint_seq2seq_path = None
checkpoint_postnet_path = None
max_decoder_steps = 500
replace_pronunciation_prob = 0.0
output_html = False
speaker_id = None
if speaker_id is not None:
	speaker_id = int(speaker_id)
preset = "/content/TTS-Makedonka/presets/deepvoice3_mkdspeech.json"

# вчитување на пресетирањата
if preset is not None:
	with open(preset) as f:
		hparams.parse_json(f.read())
assert hparams.name == "deepvoice3"

_frontend = getattr(frontend, hparams.frontend)
import train
train._frontend = _frontend
from train import plot_alignment, build_model

# Градење на моделот
model = build_model()


# Load checkpoints separately
if checkpoint_postnet_path is not None and checkpoint_seq2seq_path is not None:
	checkpoint = _load(checkpoint_seq2seq_path)
	model.seq2seq.load_state_dict(checkpoint["state_dict"])
	checkpoint = _load(checkpoint_postnet_path)
	model.postnet.load_state_dict(checkpoint["state_dict"])
	checkpoint_name = splitext(basename(checkpoint_seq2seq_path))[0]
else:
	checkpoint = _load(checkpoint_path)
	model.load_state_dict(checkpoint["state_dict"])
	checkpoint_name = splitext(basename(checkpoint_path))[0]

model.seq2seq.decoder.max_decoder_steps = max_decoder_steps

model = model.to(device)
model.eval()
model.make_generation_fast_()

In [75]:
#Синтеза на говор
text = "Здраво, моето име е Зоран."
text = text.strip();
words = nltk.word_tokenize(text)
waveform, alignment, _, _ = tts(model, text, p=replace_pronunciation_prob, speaker_id=speaker_id, fast=True)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [76]:
#Визуелизација на излезот
IPython.display.display(Audio(waveform, rate=22050))